# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os, sys
from unityagents import UnityEnvironment
import numpy as np

import torch

os.chdir('..')

sys.path.insert(0, 'continuous_control/src')

from model import (
    Critic,
    Actor,
    DDPG,
)

from util import _state_to_torch

from buffer import ReplayBuffer

from noise import OUNoise

from torch.utils.tensorboard import SummaryWriter

env = UnityEnvironment(file_name='continuous_control/unity/Reacher_Linux_Many/Reacher.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]

print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

print('The state for the first agent looks like:', states[0])

from environment_interaction import (
    evaluation,
    train,
)

%load_ext autoreload
%autoreload 2

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

## Setting up hyperparamters

In [2]:
experiement_name = "final_no_re_low_low_low_sigma"

batch_size = 128
replay_buffer_size = int(1e6)
beta = 1
beta_2 = 2

episodes = 2000

gamma = 0.99

device = "cuda"

update_steps = 45
inference_steps = 20

sigma_end = 0.1
sigma_decay = 0.86
sigma_init = 1

ou_sigma = 0.25

eval_rounds = 1

scale_reward = 10

lr_critic = 10**-4
lr_actor = 10**-4
tau_critic = 10**-3
tau_actor = 10**-3

## Setting up the networks

In [3]:
actor = Actor(
    in_dim=state_size,
    out_dim=action_size,
    hidden_dim=256,
    squeeze_dim=128,
)        

critic = Critic(
    in_dim=state_size+action_size,
    out_dim=1,
    hidden_dim=256,
    squeeze_dim=128,
)        


actor_target = actor.copy()
critic_target = critic.copy()


ddpg = DDPG(
    critic=critic,
    actor=actor,
    critic_target=critic_target,
    actor_target=actor_target,
    lr_critic=lr_critic,
    lr_actor=lr_actor,
    tau_critic=tau_critic,
    tau_actor=tau_actor,
)

## Setting up the replaybuffer
replaybuffer = ReplayBuffer(
    replay_buffer_size,
    state_size=state_size,
    action_size=action_size,
)

ddpg = ddpg.to(device)

## Train the model

In [4]:
writer = SummaryWriter(f"continuous_control/runs/{experiement_name}")

ounoise = OUNoise(
    n=len(env_info.agents),
    n_actions=action_size,
    mu=0,
    theta=0.15,
    sigma=ou_sigma,
    low=-1,
    high=1,
)

train(
    env=env,
    ddpg=ddpg,
    replaybuffer=replaybuffer,
    ounoise=ounoise, 
    writer=writer,
    episodes=episodes,
    brain_name=brain_name,
    device=device,
    batch_size=batch_size,
    gamma=gamma,
    sigma_init=sigma_init,
    sigma_end=sigma_end,
    sigma_decay=sigma_decay,
    evaluation_rounds=eval_rounds,
    train_mode=True,
    inference_steps=inference_steps,
    update_steps=update_steps,
    beta=beta,
    beta_2=beta_2,
    scale_reward=True,
)

KeyboardInterrupt: 

## Evaluate the model

In [4]:
ddpg._actor.load_state_dict(torch.load("continuous_control/models/actor.ckp"))
ddpg._critic.load_state_dict(torch.load("continuous_control/models/critic.ckp"))

score = evaluation(
    env=env,
    ddpg=ddpg,
    episodes=100,
    device=device,
    brain_name=brain_name,
    train_mode=True,
)

1 (20,) 0.0 0.0 0.0
2 (20,) 0.0 0.0 0.0
3 (20,) 0.0 0.0 0.0
4 (20,) 0.0 0.0 0.0
5 (20,) 0.0 0.0 0.0
6 (20,) 0.0 0.0 0.0
7 (20,) 0.0 0.0 0.0
8 (20,) 0.0 0.0 0.0
9 (20,) 0.0 0.0 0.0
10 (20,) 0.1099999975413084 0.0 0.03999999910593033
11 (20,) 0.3199999928474426 0.0 0.03999999910593033
12 (20,) 0.6199999861419201 0.0 0.03999999910593033
13 (20,) 0.7399999834597111 0.0 0.03999999910593033
14 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
15 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
16 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
17 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
18 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
19 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
20 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
21 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
22 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033


166 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
167 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
168 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
169 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
170 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
171 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
172 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
173 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
174 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
175 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
176 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
177 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
178 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
179 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
180 (20,) 0.7999999821186066 0.039

335 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
336 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
337 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
338 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
339 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
340 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
341 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
342 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
343 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
344 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
345 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
346 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
347 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
348 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
349 (20,) 0.7999999821186066 0.039

506 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
507 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
508 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
509 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
510 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
511 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
512 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
513 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
514 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
515 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
516 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
517 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
518 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
519 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
520 (20,) 0.7999999821186066 0.039

669 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
670 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
671 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
672 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
673 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
674 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
675 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
676 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
677 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
678 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
679 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
680 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
681 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
682 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
683 (20,) 0.7999999821186066 0.039

...

7179 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7180 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7181 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7182 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7183 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7184 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7185 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7186 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7187 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7188 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7189 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7190 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7191 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7192 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7193 (20,) 0.7999999

7353 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7354 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7355 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7356 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7357 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7358 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7359 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7360 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7361 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7362 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7363 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7364 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7365 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7366 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7367 (20,) 0.7999999

7517 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7518 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7519 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7520 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7521 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7522 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7523 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7524 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7525 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7526 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7527 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7528 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7529 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7530 (20,) 0.7999999821186066 0.03999999910593033 0.03999999910593033
7531 (20,) 0.7999999

7684 (20,) 0.7599999830126762 0.0 0.03999999910593033
7685 (20,) 0.7599999830126762 0.0 0.03999999910593033
7686 (20,) 0.7599999830126762 0.0 0.03999999910593033
7687 (20,) 0.7599999830126762 0.0 0.03999999910593033
7688 (20,) 0.7599999830126762 0.0 0.03999999910593033
7689 (20,) 0.7599999830126762 0.0 0.03999999910593033


KeyboardInterrupt: 

In [6]:
print(score)

39.43875911847502
